In [1]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
import statsmodels.api as sm
from pyearth import Earth
from sklearn.impute import KNNImputer
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split, cross_val_predict, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import BaggingRegressor,BaggingClassifier,AdaBoostRegressor,AdaBoostClassifier
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import GridSearchCV, ParameterGrid, StratifiedKFold, RandomizedSearchCV
from statsmodels.tools.tools import add_constant
import xgboost as xgb
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
import time

import warnings
warnings.filterwarnings("ignore")

## Prophet Feature Selection, so Dropping All Lag and Rolling

In [2]:
covid_df = pd.read_csv('texas_multi_df.csv', parse_dates=['Date'])
covid_df.drop(columns = ['PROVINCE_STATE_NAME'], inplace = True)
covid_df = covid_df.rename(columns={'Date': 'ds', 'PEOPLE_POSITIVE_NEW_CASES_COUNT': 'y'})
covid_df = covid_df.drop(['new_cases_1 lag', 'new_cases_2_lag', 'new_cases_3_lag', 'new_cases_4_lag', 'new_cases_7_lag', 'new_cases_30_lag', 'new_cases_50_lag', 'new_cases_150_lag', 'new_cases_350_lag', 'new_deaths_1_lag', 'new_deaths_2_lag', 'new_deaths_3_lag', 'new_deaths_4_lag', 'new_deaths_7_lag', 'new_deaths_30_lag', 'new_deaths_50_lag'], axis=1)

train_size = int(len(covid_df)* 0.85)
covid_train = covid_df[0:train_size] # notice here the original test test is never used

In [3]:
train_size

671

In [4]:
X_train = covid_train.drop(columns=['ds', 'y'], axis=1)
y_train = covid_train['y']

In [5]:
model_a = CatBoostRegressor().fit(X_train, y_train, verbose=False)
predictors_1 = pd.concat([pd.Series(model_a.feature_importances_),
           pd.Series(X_train.columns)],axis=1).sort_values(by = 0,ascending = False).head(30)
print(predictors_1[1].to_list())

['new_hospitalized_patients', 'new_cases_10_mean', 'PEOPLE_DEATH_NEW_COUNT', 'new_cases_10_max', 'current_intensive_care_patients', 'new_cases_10_std', 'new_deaths_100_std', 'relative_humidity', 'dew_point', 'new_deaths_10_mean', 'rainfall_mm', 'new_cases_50_mean', 'new_cases_50_std', 'new_deaths_10_std', 'new_cases_10_min', 'minimum_temperature_celsius', 'average_temperature_celsius', 'maximum_temperature_celsius', 'new_deaths_50_std', 'new_deaths_50_mean', 'new_deaths_10_min', 'new_cases_100_min', 'new_cases_100_mean', 'Booster_Doses', 'new_deaths_100_max', 'new_cases_100_std', 'new_deaths_100_mean', 'Booster_Doses_50Plus', 'new_cases_50_max', 'Booster_Doses_65Plus']


In [6]:
model_b = LGBMRegressor().fit(X_train, y_train, verbose=False)
predictors_2 = pd.concat([pd.Series(model_b.feature_importances_),
           pd.Series(X_train.columns)],axis=1).sort_values(by = 0,ascending = False).head(30)
print(predictors_2[1].to_list())

['new_cases_10_std', 'new_cases_10_mean', 'PEOPLE_DEATH_NEW_COUNT', 'new_hospitalized_patients', 'new_deaths_10_mean', 'new_cases_10_max', 'relative_humidity', 'new_deaths_10_std', 'dew_point', 'minimum_temperature_celsius', 'current_intensive_care_patients', 'new_cases_50_std', 'average_temperature_celsius', 'maximum_temperature_celsius', 'new_cases_10_min', 'new_deaths_50_std', 'new_cases_50_mean', 'Booster_Doses_65Plus', 'new_deaths_100_std', 'Administered_Dose1_Recip_5Plus', 'new_deaths_10_min', 'rainfall_mm', 'new_deaths_50_mean', 'new_cases_100_std', 'Series_Complete_5Plus', 'Booster_Doses', 'new_cases_100_mean', 'new_cases_50_max', 'new_deaths_10_max', 'Booster_Doses_18Plus']


In [7]:
from sklearn.ensemble import GradientBoostingRegressor,GradientBoostingClassifier, BaggingRegressor,BaggingClassifier,RandomForestRegressor,RandomForestClassifier,AdaBoostRegressor,AdaBoostClassifier

model_c = GradientBoostingRegressor().fit(X_train, y_train)
predictors_3 = pd.concat([pd.Series(model_c.feature_importances_),
           pd.Series(X_train.columns)],axis=1).sort_values(by = 0,ascending = False).head(30)
print(predictors_3[1].to_list())

['new_cases_10_mean', 'new_hospitalized_patients', 'current_intensive_care_patients', 'PEOPLE_DEATH_NEW_COUNT', 'new_cases_10_max', 'new_cases_10_std', 'new_cases_50_mean', 'new_deaths_50_std', 'new_deaths_10_mean', 'new_deaths_100_std', 'dew_point', 'Series_Complete_18Plus', 'Completeness_pct', 'new_cases_10_min', 'new_cases_100_std', 'new_deaths_10_std', 'new_cases_50_std', 'Administered_Dose1_Recip', 'new_deaths_50_mean', 'new_cases_100_mean', 'rainfall_mm', 'Booster_Doses_50Plus', 'minimum_temperature_celsius', 'vaccination_rate_1dose', 'stringency_index', 'new_deaths_10_min', 'average_temperature_celsius', 'Booster_Doses_65Plus', 'Series_Complete_5Plus', 'Administered_Dose1_Recip_5Plus']


In [8]:
from sklearn.ensemble import BaggingRegressor,BaggingClassifier,AdaBoostRegressor,AdaBoostClassifier
from sklearn.tree import DecisionTreeRegressor,DecisionTreeClassifier

model_d = AdaBoostRegressor(random_state=1, base_estimator = DecisionTreeRegressor(random_state=1)).fit(X_train, y_train)
predictors_4 = pd.concat([pd.Series(model_d.feature_importances_),
           pd.Series(X_train.columns)],axis=1).sort_values(by = 0,ascending = False).head(30)
print(predictors_4[1].to_list())

['new_cases_10_mean', 'new_cases_10_max', 'new_cases_10_std', 'new_hospitalized_patients', 'PEOPLE_DEATH_NEW_COUNT', 'current_intensive_care_patients', 'maximum_temperature_celsius', 'new_cases_10_min', 'Booster_Doses_65Plus', 'new_cases_100_mean', 'new_deaths_50_mean', 'new_cases_100_std', 'rainfall_mm', 'new_deaths_10_mean', 'Administered_Dose1_Recip', 'relative_humidity', 'Booster_Doses_50Plus', 'new_deaths_50_std', 'booster_rate', 'new_cases_50_std', 'Administered_Dose1_Recip_18Plus', 'dew_point', 'minimum_temperature_celsius', 'Series_Complete_5Plus', 'school_closing', 'new_deaths_10_std', 'average_temperature_celsius', 'vaccination_rate_complete', 'vaccination_policy', 'Series_Complete_18Plus']


In [9]:
from sklearn.ensemble import BaggingRegressor,BaggingClassifier,RandomForestRegressor,RandomForestClassifier

model_e = RandomForestRegressor().fit(X_train, y_train)
predictors_5 = pd.concat([pd.Series(model_e.feature_importances_), pd.Series(X_train.columns)], axis=1).sort_values(by=0, ascending = False).head(30)
print(predictors_5[1].to_list())

['new_cases_10_mean', 'new_hospitalized_patients', 'current_intensive_care_patients', 'PEOPLE_DEATH_NEW_COUNT', 'new_cases_10_max', 'new_cases_10_std', 'new_cases_100_std', 'new_deaths_10_mean', 'new_cases_50_std', 'new_cases_10_min', 'new_deaths_10_std', 'relative_humidity', 'new_deaths_50_mean', 'new_cases_50_mean', 'maximum_temperature_celsius', 'dew_point', 'new_cases_100_mean', 'rainfall_mm', 'Series_Complete_12Plus', 'new_deaths_100_std', 'minimum_temperature_celsius', 'new_deaths_10_min', 'vaccination_rate_complete', 'average_temperature_celsius', 'Booster_Doses_65Plus', 'new_deaths_100_mean', 'Series_Complete_5Plus', 'booster_rate', 'new_deaths_50_std', 'Series_Complete_5to17']


In [10]:
# using MARS summary for feature selection
model = Earth(max_terms=1000, max_degree=1, feature_importance_type = 'rss') # note, terms in brackets are the hyperparameters 
model.fit(X_train,y_train)
print(model.summary())

Earth Model
--------------------------------------------------------------------
Basis Function                                 Pruned  Coefficient  
--------------------------------------------------------------------
(Intercept)                                    No      85230.5      
new_cases_10_mean                              Yes     None         
new_deaths_10_mean                             No      -161.295     
h(PEOPLE_DEATH_NEW_COUNT-334)                  No      -155.353     
h(334-PEOPLE_DEATH_NEW_COUNT)                  Yes     None         
h(new_deaths_10_max-480)                       No      46.8055      
h(480-new_deaths_10_max)                       No      -16.911      
h(new_cases_50_mean-18223.5)                   Yes     None         
h(18223.5-new_cases_50_mean)                   No      -2.00675     
h(new_deaths_50_mean-258.08)                   No      -625.713     
h(258.08-new_deaths_50_mean)                   Yes     None         
h(new_cases_10_std-832

In [11]:
def common_member(a, b, c, d, e):
    set_1 = set(a)
    set_2 = set(b)
    set_3 = set(c)
    set_4 = set(d)
    set_5 = set(e)
 
    if (set_1 & set_2 & set_3 & set_4 & set_5):
        print(set_1 & set_2 & set_3 & set_4 & set_5)


In [12]:
a = predictors_1[1].to_list()
b = predictors_2[1].to_list()
c = predictors_3[1].to_list()
d = predictors_4[1].to_list()
e = predictors_5[1].to_list()

In [13]:
full = common_member(a, b, c, d, e)
full

{'new_hospitalized_patients', 'PEOPLE_DEATH_NEW_COUNT', 'minimum_temperature_celsius', 'new_cases_10_max', 'new_cases_10_mean', 'new_cases_50_std', 'new_cases_100_mean', 'dew_point', 'new_deaths_50_mean', 'new_deaths_50_std', 'current_intensive_care_patients', 'new_cases_10_min', 'new_deaths_10_std', 'average_temperature_celsius', 'new_deaths_10_mean', 'Booster_Doses_65Plus', 'new_cases_10_std', 'rainfall_mm', 'new_cases_100_std'}


## XGBoost Feature Selection, includes lag and rolling

In [ ]:
import os
os._exit(00)

In [1]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
import statsmodels.api as sm
from pyearth import Earth
from sklearn.impute import KNNImputer
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split, cross_val_predict, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import BaggingRegressor,BaggingClassifier,AdaBoostRegressor,AdaBoostClassifier
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import GridSearchCV, ParameterGrid, StratifiedKFold, RandomizedSearchCV
from statsmodels.tools.tools import add_constant
import xgboost as xgb
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
import time

import warnings
warnings.filterwarnings("ignore")

In [2]:
covid_df = pd.read_csv('texas_multi_df.csv', parse_dates=['Date'])
covid_df.drop(columns = ['PROVINCE_STATE_NAME'], inplace = True)
covid_df = covid_df.rename(columns={'Date': 'ds', 'PEOPLE_POSITIVE_NEW_CASES_COUNT': 'y'})

train_size = int(len(covid_df)* 0.85)
covid_train = covid_df[0:train_size]

X_train = covid_train.drop(columns=['ds', 'y'], axis=1)
X_train = X_train.astype('float32')
y_train = covid_train['y']

In [3]:
model_a = CatBoostRegressor().fit(X_train, y_train, verbose=False)
predictors_7 = pd.concat([pd.Series(model_a.feature_importances_),
           pd.Series(X_train.columns)],axis=1).sort_values(by = 0,ascending = False).head(40)
print(predictors_7[1].to_list())

['new_cases_7_lag', 'new_cases_10_mean', 'new_hospitalized_patients', 'new_cases_10_max', 'new_cases_1 lag', 'new_cases_10_std', 'PEOPLE_DEATH_NEW_COUNT', 'new_cases_2_lag', 'new_deaths_30_lag', 'new_deaths_2_lag', 'new_deaths_50_lag', 'new_cases_3_lag', 'new_cases_50_lag', 'new_deaths_100_std', 'new_cases_10_min', 'new_cases_4_lag', 'new_deaths_3_lag', 'new_deaths_4_lag', 'new_deaths_1_lag', 'new_cases_150_lag', 'current_intensive_care_patients', 'new_cases_50_max', 'new_cases_50_std', 'new_deaths_7_lag', 'dew_point', 'new_cases_50_mean', 'relative_humidity', 'rainfall_mm', 'new_cases_100_std', 'maximum_temperature_celsius', 'new_deaths_50_max', 'Booster_Doses_12Plus', 'new_cases_30_lag', 'new_cases_100_max', 'new_deaths_50_mean', 'new_deaths_50_std', 'new_deaths_100_mean', 'new_deaths_100_min', 'minimum_temperature_celsius', 'Booster_Doses_50Plus']


In [4]:
model_b = LGBMRegressor().fit(X_train, y_train, verbose=False)
predictors_8 = pd.concat([pd.Series(model_b.feature_importances_),
           pd.Series(X_train.columns)],axis=1).sort_values(by = 0,ascending = False).head(30)
print(predictors_8[1].to_list())

['new_cases_10_std', 'new_cases_1 lag', 'new_cases_7_lag', 'new_cases_10_mean', 'new_cases_10_max', 'PEOPLE_DEATH_NEW_COUNT', 'new_hospitalized_patients', 'new_cases_4_lag', 'new_cases_2_lag', 'new_deaths_2_lag', 'new_cases_3_lag', 'new_deaths_50_lag', 'new_deaths_7_lag', 'new_deaths_4_lag', 'new_cases_50_lag', 'relative_humidity', 'new_deaths_30_lag', 'new_deaths_1_lag', 'new_cases_150_lag', 'maximum_temperature_celsius', 'current_intensive_care_patients', 'new_deaths_3_lag', 'new_cases_50_std', 'new_cases_350_lag', 'new_cases_30_lag', 'dew_point', 'new_cases_10_min', 'new_deaths_10_std', 'minimum_temperature_celsius', 'rainfall_mm']


In [5]:
model_g = xgb.XGBRegressor().fit(X_train, y_train, verbose=False)
predictors_6 = pd.concat([pd.Series(model_g.feature_importances_),
           pd.Series(X_train.columns)],axis=1).sort_values(by = 0,ascending = False).head(30)
print(predictors_6[1].to_list())

['new_cases_10_mean', 'new_cases_10_max', 'new_cases_7_lag', 'new_hospitalized_patients', 'new_deaths_2_lag', 'contact_tracing', 'new_cases_50_max', 'new_cases_2_lag', 'new_cases_100_mean', 'new_deaths_50_lag', 'new_cases_1 lag', 'new_cases_10_std', 'new_deaths_1_lag', 'current_intensive_care_patients', 'Administered_Dose1_Recip', 'Completeness_pct', 'new_cases_350_lag', 'dew_point', 'new_deaths_30_lag', 'new_deaths_100_mean', 'PEOPLE_DEATH_NEW_COUNT', 'new_cases_10_min', 'new_cases_4_lag', 'new_deaths_10_min', 'new_cases_50_lag', 'new_deaths_7_lag', 'new_deaths_4_lag', 'new_deaths_3_lag', 'new_deaths_100_std', 'new_cases_50_std']


In [6]:
model = Earth(max_terms=1000, max_degree=1, feature_importance_type = 'rss', allow_missing=True) 
model.fit(X_train,y_train)
print(model.summary())

Earth Model
---------------------------------------------------------------------------
Basis Function                                       Pruned  Coefficient   
---------------------------------------------------------------------------
(Intercept)                                          No      587759        
new_cases_10_mean                                    No      2.08445       
present(new_deaths_2_lag)                            Yes     None          
missing(new_deaths_2_lag)                            Yes     None          
h(new_deaths_2_lag-402)*present(new_deaths_2_lag)    No      -10.68        
h(402-new_deaths_2_lag)*present(new_deaths_2_lag)    No      -15.9691      
present(new_cases_2_lag)                             No      32368.5       
missing(new_cases_2_lag)                             Yes     None          
h(new_cases_2_lag-21281)*present(new_cases_2_lag)    No      6.26914       
h(21281-new_cases_2_lag)*present(new_cases_2_lag)    Yes     None          


In [7]:
def common_member(a, b, c):
    set_1 = set(a)
    set_2 = set(b)
    set_3 = set(c)
 
    if (set_1 & set_2 & set_3):
        print(set_1 & set_2 & set_3)


In [8]:
a = predictors_7[1].to_list()
b = predictors_8[1].to_list()
c = predictors_6[1].to_list()

In [9]:
full = common_member(a, b, c)
full

{'new_cases_10_min', 'new_deaths_2_lag', 'PEOPLE_DEATH_NEW_COUNT', 'new_cases_10_std', 'new_cases_7_lag', 'new_hospitalized_patients', 'dew_point', 'new_cases_10_max', 'new_deaths_7_lag', 'new_deaths_1_lag', 'new_cases_50_lag', 'new_deaths_30_lag', 'new_cases_50_std', 'new_deaths_3_lag', 'new_cases_2_lag', 'new_cases_10_mean', 'new_cases_4_lag', 'new_cases_1 lag', 'current_intensive_care_patients', 'new_deaths_4_lag', 'new_deaths_50_lag'}
